In [125]:
import pandas as pd
import numpy as np

import datetime 
import pickle

import openpyxl


In [126]:
data = pd.read_csv("Daten_Aufgabe_1 - Shopauswertung.csv", delimiter=';')
data.tail()


,Datum,Produkt,Umsatz
1090,27.12.2021,A+B+C,197909.2416
1091,28.12.2021,A+B+C,198014.8405
1092,29.12.2021,A+B+C,198355.2760
1093,30.12.2021,A+B+C,199081.4461
1094,31.12.2021,A+B+C,199213.8531


In [127]:
#data.to_excel("Shop_eu.xlsx", engine='openpyxl', float_format='%.2f')

In [128]:
data.loc[data["Produkt"] != "A", "Umsatz"] = data.loc[data["Produkt"] != "A", "Umsatz"].diff()

In [129]:
#data = pd.read_excel("Daten_eu_sauber.xlsx", sheet_name="SAUBER")
data.tail()

,Datum,Produkt,Umsatz
1090,27.12.2021,A+B+C,858.8611
1091,28.12.2021,A+B+C,105.5989
1092,29.12.2021,A+B+C,340.4355
1093,30.12.2021,A+B+C,726.1701
1094,31.12.2021,A+B+C,132.4070


In [130]:
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y')
data['Monat'] = data['Datum'].dt.strftime('%B')
data.head(1)


,Datum,Produkt,Umsatz,Monat
0,2021-01-01,A,17.132115,January


In [131]:
data_grouped = data.groupby(['Monat', 'Produkt'])['Umsatz'].sum().reset_index()

data_pivot =data_grouped.pivot(index='Monat', columns='Produkt', values='Umsatz')
data_pivot.columns = ['Umsatz A', 'Umsatz A+B+C','Umsatz B']

data_pivot.head(3)


,Umsatz A,Umsatz A+B+C,Umsatz B
Monat,,,
April,1392.164019,16950.68751,1530.0
August,1587.995589,15693.65690,1713.0
December,1603.259023,13974.92980,1666.0


In [132]:
data_pivot["Umsatz C"] = data_pivot["Umsatz A+B+C"] - (data_pivot["Umsatz B"] + data_pivot["Umsatz A"])

data_pivot["Umsatz A"] = round(data_pivot["Umsatz A"],2)
data_pivot["Umsatz B"] = round(data_pivot["Umsatz B"],2)
data_pivot["Umsatz C"] = round(data_pivot["Umsatz C"],2)
data_pivot["Umsatz A+B+C"] = round(data_pivot["Umsatz A+B+C"],2)

data_pivot.head(3)

,Umsatz A,Umsatz A+B+C,Umsatz B,Umsatz C
Monat,,,,
April,1392.16,16950.69,1530.0,14028.52
August,1588.00,15693.66,1713.0,12392.66
December,1603.26,13974.93,1666.0,10705.67


In [133]:
data_pivot.reset_index(inplace=True)

In [134]:
data_pivot["Monat"] = pd.Categorical(data_pivot["Monat"],
                    categories=["January", "February", "March",
                    "April", "May", "June", "July", "August",
                    "September", "October", "November", "December"], ordered=True)
data_pivot = data_pivot.sort_values(by=['Monat'])


In [135]:
data_pivot.head(3)

,Monat,Umsatz A,Umsatz A+B+C,Umsatz B,Umsatz C
4,January,1514.91,17437.41,1572.0,14350.50
3,February,1495.04,12080.83,1430.0,9155.79
7,March,1789.00,16044.00,1317.0,12937.99


In [136]:
data_pivot.index = pd.RangeIndex.from_range(range(0, 12))

In [137]:
shop_eu = data_pivot[["Monat", "Umsatz A", "Umsatz B", "Umsatz C", "Umsatz A+B+C"]]
#data_pivot.head(3)
shop_eu.head(3)

,Monat,Umsatz A,Umsatz B,Umsatz C,Umsatz A+B+C
0,January,1514.91,1572.0,14350.50,17437.41
1,February,1495.04,1430.0,9155.79,12080.83
2,March,1789.00,1317.0,12937.99,16044.00


In [138]:
shop_eu = shop_eu.rename(columns={"Umsatz A":"Produkt_A_EU", "Umsatz B":"Produkt_B_EU", "Umsatz C":"Produkt_C_EU",
"Umsatz A+B+C":"Gesamt_EU"})

In [139]:
shop_eu.to_pickle("shop_eu.pkl")
shop_EU = pd.read_pickle("shop_eu.pkl")
shop_EU

,Monat,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,1514.91,1572.0,14350.50,17437.41
1,February,1495.04,1430.0,9155.79,12080.83
2,March,1789.00,1317.0,12937.99,16044.00
3,April,1392.16,1530.0,14028.52,16950.69
4,May,1719.61,1435.0,10523.91,13678.51
5,June,1588.09,1392.0,12650.49,15630.58
6,July,1813.79,1710.0,9448.17,12971.96
7,August,1588.00,1713.0,12392.66,15693.66
8,September,1495.48,1632.0,12147.95,15275.43
9,October,1836.13,1502.0,13321.29,16659.42
